In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from static_grader import grader

# Program Flow exercises

The objective of these exercises is to develop your ability to use iteration and conditional logic to build reusable functions. We will be extending our `get_primes` example from the (WQ_ProgramFlow.ipynb)[Program Flow notebook] for testing whether much larger numbers are prime. Large primes are useful for encryption. It is too slow to test every possible factor of a large number to determine if it is prime, so we will take a different approach.

## Exercise 1: `mersenne_numbers`

A Mersenne number is any number that can be written as $2^p - 1$ for some $p$. For example, 3 is a Mersenne number ($2^2 - 1$) as is 31 ($2^5 - 1$). We will see that it is easy to test if Mersenne numbers are prime.

Write a function that accepts an exponent $p$ and returns the corresponding Mersenne number. 

In [3]:
def mersenne_number(p):
    return 2**p - 1

Mersenne numbers can only be prime if their exponent, $p$, is prime. Make a list of the Mersenne numbers for all primes $p$ between 3 and 65 (there should be 17 of them).

Hint: It may be useful to modify the `is_prime` and `get_primes` functions from (WQ_ProgramFlow.ipynb)[the Program Flow notebook] for use in this problem.

In [4]:
def is_prime(number):
    if number == 1:
        return False
    
    for factor in range(2, number):
        if number % factor == 0:
            return False

    return True


my_list = []
for p in range(3,65):
    if is_prime(p) == True:
        print(p)
        my_list.append(mersenne_number(p))
        

# we can make a list like this
#my_list = [0, 1, 2]
print my_list

3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
[7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951]


In [5]:
# we can also make an empty list and add items to it
another_list = []
print another_list

for item in my_list:
    another_list.append(item)

print another_list

[]
[7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951]


In [6]:
def is_prime(number):
    if number == 1:
        return False
    
    for factor in range(2, number):
        if number % factor == 0:
            return False

    return True

def get_primes(n_start, n_end):
    result = []
    for p in range(n_start, n_end):
        if is_prime(p) == True:
            result.append(mersenne_number(p))
    return result

In [7]:
def return_mersenne():
    return get_primes(3,65)

grader.score('ip__mersenne_numbers', return_mersenne)

Your score:  1.0


## Exercise 2: `lucas_lehmer`

We can test if a Mersenne number is prime using the Lucas-Lehmer series. First let's write a function that generates the series. Given a Mersenne number with exponent $p$, the series can be defined as

$$ n_0 = 4 $$
$$ n_i = n_{i-1}^2 - 2 mod (2^p - 1) $$

Write a function that accepts the exponent $p$ of a Mersenne number and returns a list of the Lucas-Lehmer series up to $i = p - 2$.

In [8]:
def lucas_lehmer(p):
    result = [4]
    for i in range(1,p-1):
        tmp = (result[i-1]**2 - 2) % (2**p - 1)
        result.append(tmp)
    return result
lucas_lehmer(17)

[4,
 14,
 194,
 37634,
 95799,
 119121,
 66179,
 53645,
 122218,
 126220,
 70490,
 69559,
 99585,
 78221,
 130559,
 0]

Use your function to calculate the Lucas-Lehmer series for $p = 17$ and pass the result to the grader.

In [9]:
def ll_result():
    #return [4] * 16
    return lucas_lehmer(17)

grader.score('ip__lucas_lehmer', ll_result)

Your score:  1.0


## Exercise 3: `mersenne_primes`

For a given Mersenne number with exponent $p$, the number is prime if the Lucas-Lehmer series is 0 at position $p-2$. Write a function that tests if a Mersenne number with exponent $p$ is prime. Test if the Mersenne numbers with prime $p$ between 3 and 65 (i.e. 3, 5, 7, ..., 61) are prime. Your final answer should be a list of tuples consisting of `(Mersenne exponent, 0)` (or `1`) for each Mersenne number you test, where `0` and `1` are replacements for `False` and `True` respectively.

_HINT: The `zip` function is useful for combining two lists into a list of tuples_

In [10]:
def ll_prime(p):
    tmp = lucas_lehmer(p)[-1]
    if tmp == 0:
        return True
    else:
        return False
print(ll_prime(11))

False


In [11]:
def prime_test_results():
    result = []
    for p in range(3,65):
        if is_prime(p) == True:
            if ll_prime(p) == True:
                result.append((p, 1))
            else:
                result.append((p, 0))
    return result

grader.score('ip__mersenne_primes', prime_test_results)

Your score:  1.0


## Exercise 4: Optimize `is_prime`

You might have noticed that the primality check `is_prime` we developed before is somewhat slow.  This is because we are doing a ton of extra work checking modular arithmetic.  We will use two optimizations to make a `is_prime_fast` function.

The first optimization takes advantage of the fact that two is the only even prime.  Thus we can check if a number is even and as long as its greater than 2, we know that it is not prime.  

Our second optimization takes advantage of the fact that when checking factors, we only need to check up the square root of a number.  Consider a number $n$ decomposed into factors $n=ab$.  There are two cases, either $n$ is prime and without loss of generality, $a=n, b=1$ or $n$ is not prime and $a,b \neq n,1$.  In this case, if $a > \sqrt{n}$, then $b<\sqrt{n}$.  So we only need to check all possible values of $b$ and we get the values of $a$ for free!  This means that even the simple method of checking factors will increase in complexity as a square root compared to the size of the number and not linearly.

Lets write the function to do this and check the speed!  `is_prime_fast` will take a number and return whether or not it is prime.

You will see the functions followed by a cell with an `assert` statement.  These cells should run and produce no output, if they produce an error, then your function need to be modified.  Do not modify the assert statements, they are exactly as they should be!

In [57]:
import math
def is_prime_fast(number):
    if number == 0:
        result = True
    elif number == 1:
        result = False
    elif number == 2:
        result = True
    else:
        if n % 2 == 0:
            result = False
        else:
            num = 3
            while num <= math.sqrt(n):
                if n % num == 0:
                    result = False
                    break
                num += 2
            if num > math.sqrt(n):
                result = True
    return result
    

Run the following cell to make sure it finds the same primes as the original function.

In [24]:
for n in range(10000):
    assert is_prime(n) == is_prime_fast(n)

Now lets check the timing, here we will use the `%%timeit` magic which will time the execution of a particular cell.

In [ ]:
%%time
is_prime(67867967)

In [14]:
%%time
is_prime_fast(67867967)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15 µs


False

Now return a function which will find all prime numbers up to and including $n$.  We will submit to the grader the primes less than 20.

In [60]:
def get_primes_fast(n):
    result = []
    for i in range(2,n+1):
        #if is_prime_fast(i) == True:
        if is_prime(i) == True:
            result.append(i)
    return result
is_prime(4)
#is_prime_fast(4)
#get_primes_fast(5)

False

In [61]:
grader.score('ip__is_prime_fast', lambda : get_primes_fast(20))

Your score:  1.0


## Exercise 5: sieve

 In this problem we will develop an even faster method which is known as the Sieve of Eratosthenes (although it will be more expensive in terms of memory). The Sieve of Eratosthenes is an example of dynamic programming, where the general idea is to not redo computations we have already done (read more about it [here](https://en.wikipedia.org/wiki/Dynamic_programming)).  We will break this sieve down into several small functions. 

Our submission will be a list of all prime numbers less than 2000.

The method works as follows (see [here](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) for more details)

1. Generate a list of all numbers between 2 and N
2. Starting with $p=2$ (the first prime) mark all numbers of the form $np$ where $n>1$ and $np <= N$ to be not prime (they can't be prime since they are multiples of 2!)
3. Find the smallest number greater than $p$ which is not marked and set that equal to $p$, then go back to step 2.  Stop if there is no unmarked number greater than $p$ and less than $N+1$

We will break this up into a few functions, our general strategy will be to use a Python `list` as our container although we could use other data structures.  The index of this list will represent numbers.

We have implemented a `sieve` function which will find all the prime numbers up to $n$.  You will need to implement the functions which it calls.  They are as follows

* `list_true` Make a list of true values of length $n+1$ where the first two values are false
* `mark_false` takes a list of booleans and a number $p$.  Mark all elements $2p,3p,...n$ false.
* `find_next` Find the smallest `True` element in a list which is greater than some $p$ (has index greater than $p$.
* `prime_from_list` Return indices of True values.

Remember that python lists are zero indexed.

In [ ]:
def list_true(n):
    return ...

In [ ]:
assert len(list_true(20)) == 21
assert list_true(20)[0] is False
assert list_true(20)[1] is False

Now we want to write a function which takes a list of elements and a number $p$ and marks elements false which are in the range $2p,3p ... N$.

In [ ]:
def mark_false(bool_list, p):
    return ...

In [ ]:
assert mark_false([True, True, True, True, True], 2) == [True, True, True, True, False]

Now lets write a `find_next` function which returns the smallest element in a list which is not false and is greater than $p$.

In [ ]:
def find_next(bool_list, p):
    return ...

In [ ]:
assert find_next([True, True, True, True], 2) == 3
assert find_next([True, True, True, False], 2) is None

Now given a list of `True` and `False`, return the index of the true values.

In [ ]:
def prime_from_list(bool_list):
    return ...

In [ ]:
assert prime_from_list([False, False, True]) ==  [2]

In [ ]:
def sieve(n):
    bool_list = list_true(n)
    p = 2
    while p is not None:
        bool_list = mark_false(bool_list, p)
        p = find_next(bool_list, p)
    return prime_from_list(bool_list)

In [ ]:
%%timeit 
sieve(5000)

In [ ]:
%%timeit 
get_primes(0, 5000)

Find the first 2000 primes with the sieve and submit that to the grader.

In [ ]:
grader.score('ip__eratosthenes', lambda : sieve(2000))